In [ ]:
# from classification.ipynb
%store -r df1
# from consolidate_classes.ipynb
%store -r df

In [ ]:
df1.iloc[0, 7]

In [ ]:
from openai import OpenAI

client = OpenAI()

error_log = []


def get_embedding(text, index, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    try:
        embedding = (
            client.embeddings.create(input=[text], model=model).data[0].embedding
        )
        return embedding
    except Exception as e:
        # Log the error and the index of the problematic row
        error_log.append((index, str(e)))
        return None  # Return None for rows where the embedding generation fails

In [ ]:
df_emb["index"] = df_emb.index
df_emb["embeddings"] = df_emb.apply(
    lambda row: get_embedding(row["firm_descript"], row.name), axis=1
)

In [ ]:
df_emb = df1.copy()
df_emb["index"] = df_emb.index
df_emb["embeddings"] = df_emb.apply(
    lambda row: get_embedding(row["firm_descript"], row.name), axis=1
)

In [ ]:
error_log1 = error_log

In [ ]:
class_descriptions = {
    "Assembly, Packaging & Interconnects": "Specializes in surface-mount technology and encapsulation for robust electronic devices. Offers advanced solutions in ball-grid-array packaging, flip-chip bonding, and wire-bonding techniques. Expertise in lead-frame engineering, die-attach processes, and underfill materials to enhance durability. Provides overmolding services for mechanical protection and package-sealing to ensure device integrity.",
    "Lithography, Photomasks & Imaging": "A leader in photolithography and EUV (Extreme Ultraviolet) technology, providing state-of-the-art steppers and photomasks for precise circuit patterning. Specializes in immersion lithography techniques, reticle design, and photoresist applications. Offers aligner systems and developer solutions tailored for high-resolution exposure and imaging needs.",
    "Logic Chip Design & Software": "Focuses on EDA (Electronic Design Automation) tools and VHDL/Verilog programming for efficient logic chip design. Specializes in synthesis, SoC (System on Chip) integration, and FPGA (Field-Programmable Gate Array) development. Provides comprehensive layout and simulation services, excelling in gate-array design and RTL (Register-Transfer Level) optimization.",
    "Material & Wafer Fabrication": "Engaged in high-purity silicon-ingot production using the Czochralski process and epitaxy for semiconductor layering. Specializes in wafer-slicing, doping techniques, and polysilicon manufacturing. Offers monocrystalline silicon boules, annealing services to relieve material stress, and sputtering for thin-film deposition.",
    "Planarization, Inspection & Metrology": "Expert in CMP (Chemical Mechanical Planarization) for wafer surface preparation and defect-inspection using SEM (Scanning Electron Microscopy). Provides metrology solutions with profilometer, ellipsometry, and AFM (Atomic Force Microscopy) technologies. Specializes in nanotopography analysis, overlay measurement for layer alignment, and scatterometry for film thickness.",
}

In [ ]:
import pandas as pd

# Data for each class
data = {
    "class_name": [
        "Assembly, Packaging & Interconnects",
        "Lithography, Photomasks & Imaging",
        "Logic Chip Design & Software",
        "Material & Wafer Fabrication",
        "Planarization, Inspection & Metrology",
    ],
    "description": [
        "Specializes in surface-mount technology and encapsulation for robust electronic devices. Offers advanced solutions in ball-grid-array packaging, flip-chip bonding, and wire-bonding techniques. Expertise in lead-frame engineering, die-attach processes, and underfill materials to enhance durability. Provides overmolding services for mechanical protection and package-sealing to ensure device integrity.",
        "A leader in photolithography and EUV (Extreme Ultraviolet) technology, providing state-of-the-art steppers and photomasks for precise circuit patterning. Specializes in immersion lithography techniques, reticle design, and photoresist applications. Offers aligner systems and developer solutions tailored for high-resolution exposure and imaging needs.",
        "Focuses on EDA (Electronic Design Automation) tools and VHDL/Verilog programming for efficient logic chip design. Specializes in synthesis, SoC (System on Chip) integration, and FPGA (Field-Programmable Gate Array) development. Provides comprehensive layout and simulation services, excelling in gate-array design and RTL (Register-Transfer Level) optimization.",
        "Engaged in high-purity silicon-ingot production using the Czochralski process and epitaxy for semiconductor layering. Specializes in wafer-slicing, doping techniques, and polysilicon manufacturing. Offers monocrystalline silicon boules, annealing services to relieve material stress, and sputtering for thin-film deposition.",
        "Expert in CMP (Chemical Mechanical Planarization) for wafer surface preparation and defect-inspection using SEM (Scanning Electron Microscopy). Provides metrology solutions with profilometer, ellipsometry, and AFM (Atomic Force Microscopy) technologies. Specializes in nanotopography analysis, overlay measurement for layer alignment, and scatterometry for film thickness.",
    ],
}

# Creating a DataFramec
classes = pd.DataFrame(data)

classes

In [ ]:
classes["embeddings"] = classes.apply(
    lambda row: get_embedding(row["description"], row.name), axis=1
)

In [ ]:
classes

In [ ]:
df_emb

In [ ]:
t = "company headquartered republic korea engaged manufacture trade consumer electronics mobile communication device founded produce electronic appliance tv calculator refrigerator air conditioner washer digital medium business area cover computer device laptop computer laser printer video display television computer monitor consumer entertainment device dvd player player digital camcorder semiconductor business area includes semiconductor chip sdram sram nand flash memory smart card mobile application processor mobile tv receiver rf transceivers cmos image sensor smart card ic ic player soc package mcp storage device optical disc drive hard disk drive telecommunication network business area includes dslams fax machine cellular device mobile phone pda phone hybrid device called mobile intelligent terminal mit satellite receiver lcd business area focus panel laptop monitor television digital appliance business area includes home appliance refrigerator air conditioner air purifier washer microwave oven vacuum cleaner brand include anycall mobile phone sen laptop computer hauzen premium home appliance zipel home refrigerator yepp player pavv tv kenox digimax digital camera syncmaster monitor magic station pc ultra edition slim mobile phone pleomax pc peripheral battery nexus player xm satellite strives become beloved brand innovative company admired company company business consists ce tv monitor printer air conditioner refrigerator im mobile phone communication system computer d memory semiconductor system lsi company global competitiveness listed company samsung c samsung sdi company maintains stable profitability thanks solid business portfolio secured global competitiveness tv smartphone semiconductor display panel division consumer electronics mobile communication device source bureau van dijk consumer electronics mobile "

In [ ]:
em = get_embedding(t, 0)

In [ ]:
len(em)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Iterate through each row in df_emb
for index, row in df_emb.iterrows():
    try:
        # Convert firm embeddings to numpy array and reshape
        firm_embedding = np.array(row["embeddings"]).reshape(1, -1)

        for _, class_row in classes.iterrows():
            try:
                # Convert class embeddings to numpy array and reshape
                class_embedding = np.array(class_row["embeddings"]).reshape(1, -1)

                # Calculate similarity
                similarity = cosine_similarity(firm_embedding, class_embedding)[0][0]
            except ValueError:
                # Handle case where class embeddings contain NaN or are invalid
                similarity = (
                    0  # or np.nan, depending on how you want to handle this case
                )

            # Update df_emb with the calculated similarity
            df_emb.at[index, f'similarity_{class_row["class_name"]}'] = similarity
    except ValueError:
        # Handle case where firm embeddings contain NaN or are invalid
        # Set similarity to 0 (or np.nan) for all classes for this firm
        for class_name in classes["class_name"]:
            df_emb.at[index, f"similarity_{class_name}"] = np.nan

# df_emb will now have updated similarity columns, handling NaN values gracefully

In [ ]:
df_emb

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

# Define the class columns and their corresponding similarity columns
class_columns = [
    "Assembly, Packaging & Interconnects",
    "Lithography, Photomasks & Imaging",
    "Logic Chip Design & Software",
    "Material & Wafer Fabrication",
    "Planarization, Inspection & Metrology",
]
similarity_columns = [
    "similarity_Assembly, Packaging & Interconnects",
    "similarity_Lithography, Photomasks & Imaging",
    "similarity_Logic Chip Design & Software",
    "similarity_Material & Wafer Fabrication",
    "similarity_Planarization, Inspection & Metrology",
]

# Initialize lists to store true positives, false positives, true negatives, and false negatives
tp, fp, tn, fn = 0, 0, 0, 0

# Iterate over each row in your DataFrame
for index, row in df_emb.iterrows():
    # For each class, check if the predicted class (highest similarity) is indeed a class of the company
    max_similarity_index = np.argmax(row[similarity_columns])
    predicted_class = class_columns[max_similarity_index]
    actual_class = row[predicted_class]

    # Update tp, fp, tn, fn based on actual_class and prediction
    if actual_class == 1:
        tp += 1  # True positive: Predicted class is correct
    else:
        fp += 1  # False positive: Predicted class is incorrect

    # For classes not predicted, update tn or fn accordingly
    for i, class_col in enumerate(class_columns):
        if i != max_similarity_index:  # Skip the predicted class
            if row[class_col] == 0:
                tn += 1  # True negative: Correctly not predicted
            else:
                fn += 1  # False negative: Incorrectly not predicted

# Calculate metrics
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

accuracy, f1, precision, recall